In [2]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Set up data paths and explore dataset
train_dir = "C:\Projects\Autoimmune\Training"
test_dir = "C:\Projects\Autoimmune\Testing"

# Function to analyze dataset structure
def analyze_dataset(train_path, test_path):
    print("Dataset Structure Analysis:")
    print("-" * 50)
    
    # Analyze training data
    print("\nTraining Dataset:")
    train_classes = os.listdir(train_path)
    for class_name in train_classes:
        class_path = os.path.join(train_path, class_name)
        n_samples = len(os.listdir(class_path))
        print(f"{class_name}: {n_samples} images")
    
    # Analyze testing data
    print("\nTesting Dataset:")
    test_classes = os.listdir(test_path)
    for class_name in test_classes:
        class_path = os.path.join(test_path, class_name)
        n_samples = len(os.listdir(class_path))
        print(f"{class_name}: {n_samples} images")

# Execute analysis
try:
    analyze_dataset(train_dir, test_dir)
except FileNotFoundError:
    print("Error: Please ensure your Training and Testing folders are in the current working directory")
    print("Current working directory:", os.getcwd())


Dataset Structure Analysis:
--------------------------------------------------

Training Dataset:
0Normal: 444 images
1Doubtful: 431 images
2Mild: 183 images
3Moderate: 167 images
4Severe: 198 images

Testing Dataset:
0Normal: 8 images
1Doubtful: 9 images
2Mild: 8 images
3Moderate: 8 images
4Severe: 8 images


In [5]:
# Set up image parameters and data generators
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Verify data paths and class names
class_names = ['0Normal', '1Doubtful', '2Mild', '3Moderate', '4Severe']
train_dir = 'C:\Projects\Autoimmune\Training'
test_dir = 'C:\Projects\Autoimmune\Testing'

# Verify folder structure
print("Verifying folder structure...")
for folder in [train_dir, test_dir]:
    for class_name in class_names:
        path = os.path.join(folder, class_name)
        if os.path.exists(path):
            num_images = len(os.listdir(path))
            print(f"{folder}/{class_name}: {num_images} images")
        else:
            print(f"Warning: {path} not found")

# Create data generators with appropriate augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators with explicit class mapping
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    classes=class_names,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    classes=class_names,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    classes=class_names,
    class_mode='categorical',
    shuffle=False
)

# Print generator details
print("\nGenerator Details:")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print("\nClass mapping:", train_generator.class_indices)


Verifying folder structure...
C:\Projects\Autoimmune\Training/0Normal: 444 images
C:\Projects\Autoimmune\Training/1Doubtful: 431 images
C:\Projects\Autoimmune\Training/2Mild: 183 images
C:\Projects\Autoimmune\Training/3Moderate: 167 images
C:\Projects\Autoimmune\Training/4Severe: 198 images
C:\Projects\Autoimmune\Testing/0Normal: 8 images
C:\Projects\Autoimmune\Testing/1Doubtful: 9 images
C:\Projects\Autoimmune\Testing/2Mild: 8 images
C:\Projects\Autoimmune\Testing/3Moderate: 8 images
C:\Projects\Autoimmune\Testing/4Severe: 8 images
Found 1141 images belonging to 5 classes.
Found 282 images belonging to 5 classes.
Found 41 images belonging to 5 classes.

Generator Details:
Training samples: 1141
Validation samples: 282
Test samples: 41

Class mapping: {'0Normal': 0, '1Doubtful': 1, '2Mild': 2, '3Moderate': 3, '4Severe': 4}


In [11]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def create_custom_cnn():
    # Define input layer explicitly
    inputs = Input(shape=(224, 224, 3))
    
    # First Convolutional Block
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Second Convolutional Block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Third Convolutional Block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    # Dense Layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(5, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [14]:
# Create and compile the model (assuming model is already defined)
model = create_custom_cnn()
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    )
]

# Train the model without the problematic parameters
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.3140 - loss: 3.3291 - val_accuracy: 0.3050 - val_loss: 1.5491 - learning_rate: 0.0010
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.3082 - loss: 1.5451 - val_accuracy: 0.1986 - val_loss: 1.5352 - learning_rate: 0.0010
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.3384 - loss: 1.5173 - val_accuracy: 0.2872 - val_loss: 1.5558 - learning_rate: 0.0010
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.3695 - loss: 1.5103 - val_accuracy: 0.2589 - val_loss: 1.5345 - learning_rate: 0.0010
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.3379 - loss: 1.4964 - val_accuracy: 0.2518 - val_loss: 1.5348 - learning_rate: 0.0010
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.3654 - loss: 1.5021 - val_accuracy: 0.2801 - val_loss: 1.5394 - learning_rate: 0.0010
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.3647 - loss: 1.4690 - val_accuracy:

In [15]:
def create_improved_cnn():
    inputs = Input(shape=(224, 224, 3))
    
    # Add batch normalization after input
    x = BatchNormalization()(inputs)
    
    # Increase initial filters and add residual connections
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.3)(x)
    
    # Additional layers with increased capacity
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(5, activation='softmax')(x)
    
    return Model(inputs, outputs)


In [16]:
# Modify training parameters
model.compile(
    optimizer=Adam(learning_rate=0.0005),  # Lower initial learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Update callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
]


In [19]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [21]:
from tensorflow.keras.applications import DenseNet121

def create_transfer_model():
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model
    base_model.trainable = False
    
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(5, activation='softmax')
    ])
    return model


In [ ]:
# Calculate class weights to handle imbalance
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Update training
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks=callbacks,
    class_weight=class_weight_dict
)


In [ ]:
# Implement a warmup learning rate schedule
initial_learning_rate = 1e-4
warmup_epochs = 5

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate,
    decay_steps=20*len(train_generator),
    alpha=1e-6
)

optimizer = Adam(learning_rate=lr_schedule)
